In [203]:
!ls '../'

input  lib  working


In [204]:
!ls '../input/'

coco-2017-dataset    thesis-imgcap-imgencodings-1
embeddings-glove200  thesis-imgcap-run2-deterministic-info


In [205]:
!ls '../input/'

coco-2017-dataset    thesis-imgcap-imgencodings-1
embeddings-glove200  thesis-imgcap-run2-deterministic-info


In [206]:
!ls '../input/thesis-imgcap-run2-deterministic-info/'

descriptions_test.pkl		       img_encodings_train_97000.pkl
descriptions_train_97000.pkl	       img_encodings_val_3000.pkl
descriptions_train_97000_startend.pkl  ixtoword_train_97000.pkl
descriptions_train_and_val.pkl	       train_imgs_keys_97000_randomSplit444.pkl
descriptions_val_3000.pkl	       val_imgs_keys_3000_randomSplit444.pkl
embedding_matrix_6758_200.pkl	       wordtoix_train_97000.pkl


In [207]:
!ls '../input/coco-2017-dataset/coco2017/train2017/' | wc -l

118287


In [208]:
!ls '../input/coco-2017-dataset/coco2017/val2017/' | wc -l

5000


In [209]:
!ls '../input/coco-2017-dataset/coco2017/annotations/'

captions_train2017.json   instances_val2017.json
captions_val2017.json	  person_keypoints_train2017.json
instances_train2017.json  person_keypoints_val2017.json


In [210]:
!ls '../input/thesis-imgcap-imgencodings-1/'

train2017_subset_5k_0_5000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_10000_15000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_15000_20000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_20000_25000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_25000_30000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_30000_35000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_35000_40000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_40000_45000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_45000_50000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_50000_55000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_5000_10000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_55000_60000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_60000_65000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_65000_70000_images_encoded_features_pickled_1.pkl
train2017_subset_5k_70000_

In [211]:
!ls '../input/thesis-imgcap-imgencodings-1/' | wc -l

21


In [212]:
!ls '../input/embeddings-glove200/'

glove200_embeddings_dict_1.pkl


In [213]:
!ls '../working/'

__notebook_source__.ipynb


In [214]:
!mkdir '../working/weights_out/'

In [215]:
!ls '../working/'

__notebook_source__.ipynb  weights_out


In [216]:
!ls '../working/weights_out/' | wc -l

0


In [202]:
#!rm -r '../working/weights_out/'

# Training model on 97k of some 100k images of Coco_Train2017 dataset. Using the balance 3k as the Validation Set. Using all the 5k images of Coco_Val2017 dataset as the Test dataset.

## Created subsets of the total data and used them to make the image feature encodings during preprocessing to avoid bottlenecking during training.

### Had already uploaded all 118287 images to Google drive.
### Made folders for subsets of 5k images.
### Split done using the list from os.listdir(gdrive folder)[ slice_start : slice_end ].
### Ran those subsets of 5k images through the CNN-Encoder to get one file of encodings of the image feature arrays.
### The encoder is pre-trained Google Inception-v3 trained on Imagenet.

## AVAILABLE data :
### Coco_Train2017     = has 118287 images
### Coco_Val2017       = has 5000   images
### Combined total     = 123287     images

## USED data :
### Coco_Train2017     = 100000  images
### Coco_Val2017       = 5000    images
### Combined total     = 105000  images

### Using only the first 100k images of Train2017 + all 5k images of Val2017
### Thus total data available for training = 100k + 5k = 105k
### Details of split of data:
### Training   data = 97000 images from Coco_Train2017
### Validation data = 3000  images from Coco_Train2017
### Test       data = 5000  images from Coco_Val2017

### Note: 1) Each image will have multiple captions (up to 5 as some may be discarded)
###       2) Not using the Coco_Test2017 dataset at all as it has no annotations json file which has the captions.

## Using this LINK: https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8
## Image captioning with Keras

In [217]:
import os
import sys
import json
import time
import datetime
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import re
import pickle
#import itertools
import PIL
import PIL.Image

## import numpy as np
## from numpy import array
## import pandas as pd
## import matplotlib.pyplot as plt
## %matplotlib inline
## import string
## import os
## from PIL import Image
## import glob
## from pickle import dump, load
## from time import time
## from keras.preprocessing import sequence
## from keras.models import Sequential
## from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
## from keras.optimizers import Adam, RMSprop
## from keras.layers.wrappers import Bidirectional
## from keras.layers.merge import add
## from keras.applications.inception_v3 import InceptionV3
## from keras.preprocessing import image
## from keras.models import Model
## from keras import Input, layers
## from keras import optimizers
## from keras.applications.inception_v3 import preprocess_input
## from keras.preprocessing.text import Tokenizer
## from keras.preprocessing.sequence import pad_sequences
## from keras.utils import to_categorical

In [ ]:
#from google.colab import drive
#drive.flush_and_unmount()
#drive.mount('/content/gdrive')

In [218]:
## Kaggle versions

## Weights from training till now
OPDIR = r'../working/'

## New weights to save here
OPDIR_WEIGHTS = r'../working/weights_out/'

## Images locations
IPDIR_IMGS_COCO_TRAIN = r'../input/coco-2017-dataset/coco2017/train2017/'
IPDIR_IMGS_COCO_VAL = r'../input/coco-2017-dataset/coco2017/val2017/'

## Annotations json file location from where to pick up the captions
IPDIR_ANNO = r'../input/coco-2017-dataset/coco2017/annotations/'

## Bottleneck CNN Encoder output for all the images to be used in training
IPDIR_IMG_ENCODINGS = r'../input/thesis-imgcap-imgencodings-1/'

## using GloVe - 200 file, already created the embedding matrix and pickled
IPDIR_EMBED_DICT = r'../input/embeddings-glove200/'

## to make deterministic - saved all the variables based on earlier split
IPDIR_DETERMINISTIC = '../input/thesis-imgcap-run2-deterministic-info/'


#WORD_EMBEDDINGS_DIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/WordEmbeddings/' #not needed as already created the embedding matrix and pickled

## Google drive versions
#OPDIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/ImgCapTraining/CocoVal2017_1/'
#IPDIRIMGS = '/content/gdrive/My Drive/ThesisStoryGen/Data/coco_val2017_wget_1/val2017/'
#IPDIRANNO = '/content/gdrive/My Drive/ThesisStoryGen/Data/coco_annotations_trainval2017/'
#WORD_EMBEDDINGS_DIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/WordEmbeddings/'

## ALWAYS - Data Preprocessing — Images - RELOAD FROM PICKLE FILE: Image encodings info obtained from encoder

### Create two encodings dicts from the already picked files of encodings:
###   1) img_encodings_train_and_val from the Train2017 dataset - used for the training and validation data DURING model training.
###      Will contain 100k image feature vectors
###   2) img_encodings_test from the Val2017   dataset - used only for the Testing phase AFTER model training is completed.
###      Will contain 5k image feature vectors

### e.g. a val2017 dataset image entry is: img_encodings['000000179765'] should be as below:
#### array([0.14290808, 0.14481388, 0.30199888, ..., 0.20583029, 0.1378399 ,
####        0.05842396], dtype=float32)

In [219]:
os.listdir(IPDIR_IMG_ENCODINGS)

['train2017_subset_5k_55000_60000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_70000_75000_images_encoded_features_pickled_1.pkl',
 'val2017_all_5k_images_encoded_features_pickled_2.pkl',
 'train2017_subset_5k_95000_100000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_40000_45000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_60000_65000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_35000_40000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_30000_35000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_25000_30000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_45000_50000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_5000_10000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_85000_90000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_65000_70000_images_encoded_features_pickled_1.pkl',
 'train2017_subset_5k_15000_20000_images_encoded_featur

In [220]:
## From Train2017 images pickled files
print(f"\n\nLoading encodings for the train+val images")
pickled_encodings_files_train2017 = os.listdir(IPDIR_IMG_ENCODINGS)
pickled_encodings_files_train2017.remove('val2017_all_5k_images_encoded_features_pickled_2.pkl')  ## remove the val2017 pickle file
img_encodings_train_and_val = {}
for idx, subset_encodings_file_train2017 in enumerate(pickled_encodings_files_train2017):
    with open(IPDIR_IMG_ENCODINGS + subset_encodings_file_train2017, 'rb') as handle:
        img_encodings_train_and_val.update(pickle.load(handle))
        print(f"Loaded file num {idx+1} :: {subset_encodings_file_train2017}")
    print(f"Number of entries in img_encodings_train2017 = {len(img_encodings_train_and_val)}")
print(f"\nFinal count img_encodings_train_and_val = {len(img_encodings_train_and_val)}. Should be = 100k.")



## From Val2017 images pickled files
print(f"\n\nLoading encodings for the test images")
pickled_encodings_files_val2017 = ['val2017_all_5k_images_encoded_features_pickled_2.pkl']
img_encodings_test = {}
for idx, subset_encodings_file_val2017 in enumerate(pickled_encodings_files_val2017):
    with open(IPDIR_IMG_ENCODINGS + subset_encodings_file_val2017, 'rb') as handle:
        img_encodings_test.update(pickle.load(handle))
        print(f"Loaded file num {idx+1} :: {subset_encodings_file_val2017}")
    print(f"Number of entries in img_encodings_test = {len(img_encodings_test)}")
print(f"\nFinal count img_encodings_test = {len(img_encodings_test)}. Should be = 5k.")

del pickled_encodings_files_train2017, pickled_encodings_files_val2017



Loading encodings for the train+val images
Loaded file num 1 :: train2017_subset_5k_55000_60000_images_encoded_features_pickled_1.pkl
Number of entries in img_encodings_train2017 = 5000
Loaded file num 2 :: train2017_subset_5k_70000_75000_images_encoded_features_pickled_1.pkl
Number of entries in img_encodings_train2017 = 10000
Loaded file num 3 :: train2017_subset_5k_95000_100000_images_encoded_features_pickled_1.pkl
Number of entries in img_encodings_train2017 = 15000
Loaded file num 4 :: train2017_subset_5k_40000_45000_images_encoded_features_pickled_1.pkl
Number of entries in img_encodings_train2017 = 20000
Loaded file num 5 :: train2017_subset_5k_60000_65000_images_encoded_features_pickled_1.pkl
Number of entries in img_encodings_train2017 = 25000
Loaded file num 6 :: train2017_subset_5k_35000_40000_images_encoded_features_pickled_1.pkl
Number of entries in img_encodings_train2017 = 30000
Loaded file num 7 :: train2017_subset_5k_30000_35000_images_encoded_features_pickled_1.pkl


## ALWAYS - create "descriptions" dictionaries mapping an image to its list of captions
## A descriptions dict has the key   = image name without the .jpg extension
##                         the value = [list of 5 captions for that image]

### Uses the annotations json files for COCO_Val2017 and COCO_Train2017

### Create two dicts:
###   1) "descriptions_train_and_val_dataset" dict from the Train2017 captions file - used for training and validation data DURING model training.
###      Will contain 118k images info when first loaded.
###      As only 100k of this is being used, will cull it to the correct 100k images and their descriptions later.
###   2) "descriptions_test_dataset" from the Val2017 captions file - used only for the Testing phase AFTER model training is completed.
###      Will contain 5k images info

### process the captions_train2017.json file - will be used to create our TRAIN AND VALIDATION DATASET of 97k and 3k images respectively.
### For now, it will have 100k values, but later it will be split using the train_test_split function of scikit_learn

In [221]:
os.listdir(IPDIR_ANNO)

['person_keypoints_train2017.json',
 'instances_val2017.json',
 'instances_train2017.json',
 'person_keypoints_val2017.json',
 'captions_val2017.json',
 'captions_train2017.json']

In [222]:
## build the descriptions_train_and_val_dataset dict

with open(IPDIR_ANNO+'captions_train2017.json', 'r') as f:
  data = json.load(f)
  #data.keys() # dict_keys(['info', 'licenses', 'images', 'annotations'])
  #type(data['annotations']) # is a list
  #type(data['images'])      # also is a list

dfanno = pd.DataFrame(data=data['annotations'])
# dfanno.columns = Index(['image_id', 'id', 'caption'], dtype='object') 
# dfanno.dtypes =
#   image_id     int64
#   id           int64
#   caption     object
#   dtype: object

dfimages = pd.DataFrame(data=data['images'])
# dfimages.columns = Index(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'], dtype='object')
# dfimages.dtypes =
#   license           int64
#   file_name        object
#   coco_url         object
#   height            int64
#   width             int64
#   date_captured    object
#   flickr_url       object
#   id                int64
#   dtype: object
## of above, am dropping useless columns
dfimages.drop(['license', 'coco_url', 'date_captured', 'flickr_url'], axis = 1, inplace=True)

## columns remaining in the dfs are:
# dfanno columns are      image_id , id , caption
#                         179765   , 38 ,	A black Honda motorcycle parked in front of a ...
# dfimages columns are    file_name        , height ,  width , id
#                         000000397133.jpg , 427    ,  640   , 397133

## the captions are not ordered for each image and seem to be randomly placed

dfanno.head(3)

,image_id,id,caption
0,203564,37,A bicycle replica with a clock as the front wh...
1,322141,49,A room with blue walls and a white sink and door.
2,16977,89,A car that seems to be parked illegally behind...


In [223]:
## marrying the "annotations" and the "images" keys info together into dfanno
## the file_name column has the actual image name from the "image" key section
dfanno = dfanno.merge(dfimages, how="inner", left_on='image_id', right_on='id')
dfanno.drop(['height', 'width', 'id_y'], axis = 1, inplace=True)
dfanno.rename(columns={'id_x':'id'}, inplace=True)
dfanno.head(3)

,image_id,id,caption,file_name
0,203564,37,A bicycle replica with a clock as the front wh...,000000203564.jpg
1,203564,181,The bike has a clock as a tire.,000000203564.jpg
2,203564,478,A black metal bicycle with a clock inside the ...,000000203564.jpg


In [224]:
## Mapping image with captions using dictionary

def create_descriptions_dictionary(_dfin):
    descriptions = {}
    for row in _dfin.itertuples():
      rowdict = row._asdict()
      img_filename = rowdict['file_name'].split('.')[0] # drop the .jpg part
      img_caption = rowdict['caption']
      if(img_filename not in descriptions):
        descriptions[img_filename] = [img_caption]
      else:
        descriptions[img_filename].append(img_caption)
    return descriptions

descriptions_train_and_val = create_descriptions_dictionary(dfanno)
print(f"len(descriptions_train_and_val) = {len(descriptions_train_and_val)}")

del dfanno

len(descriptions_train_and_val) = 118287


### process the captions_val2017.json file - will be used to create our TEST DATASET of 5k images

In [225]:
## build the descriptions_test_dataset dict

with open(IPDIR_ANNO+'captions_val2017.json', 'r') as f:
  data = json.load(f)
  #data.keys() # dict_keys(['info', 'licenses', 'images', 'annotations'])
  #type(data['annotations']) # is a list
  #type(data['images'])      # also is a list

dfanno = pd.DataFrame(data=data['annotations'])
# dfanno.columns = Index(['image_id', 'id', 'caption'], dtype='object') 
# dfanno.dtypes =
#   image_id     int64
#   id           int64
#   caption     object
#   dtype: object

dfimages = pd.DataFrame(data=data['images'])
# dfimages.columns = Index(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'], dtype='object')
# dfimages.dtypes =
#   license           int64
#   file_name        object
#   coco_url         object
#   height            int64
#   width             int64
#   date_captured    object
#   flickr_url       object
#   id                int64
#   dtype: object
## of above, am dropping useless columns
dfimages.drop(['license', 'coco_url', 'date_captured', 'flickr_url'], axis = 1, inplace=True)

## columns remaining in the dfs are:
# dfanno columns are      image_id , id , caption
#                         179765   , 38 ,	A black Honda motorcycle parked in front of a ...
# dfimages columns are    file_name        , height ,  width , id
#                         000000397133.jpg , 427    ,  640   , 397133

## the captions are not ordered for each image and seem to be randomly placed

dfanno.head(3)

,image_id,id,caption
0,179765,38,A black Honda motorcycle parked in front of a ...
1,179765,182,A Honda motorcycle parked in a grass driveway
2,190236,401,An office cubicle with four different types of...


In [226]:
## marrying the "annotations" and the "images" keys info together into dfanno
## the file_name column has the actual image name from the "image" key section
dfanno = dfanno.merge(dfimages, how="inner", left_on='image_id', right_on='id')
dfanno.drop(['height', 'width', 'id_y'], axis = 1, inplace=True)
dfanno.rename(columns={'id_x':'id'}, inplace=True)
dfanno.head(3)

,image_id,id,caption,file_name
0,179765,38,A black Honda motorcycle parked in front of a ...,000000179765.jpg
1,179765,182,A Honda motorcycle parked in a grass driveway,000000179765.jpg
2,179765,479,A black Honda motorcycle with a dark burgundy ...,000000179765.jpg


In [227]:
## Mapping image with captions using dictionary

def create_descriptions_dictionary(_dfin):
    descriptions = {}
    for row in _dfin.itertuples():
      rowdict = row._asdict()
      img_filename = rowdict['file_name'].split('.')[0] # drop the .jpg part
      img_caption = rowdict['caption']
      if(img_filename not in descriptions):
        descriptions[img_filename] = [img_caption]
      else:
        descriptions[img_filename].append(img_caption)
    return descriptions

descriptions_test = create_descriptions_dictionary(dfanno)
print(f"len(descriptions_test) = {len(descriptions_test)}")

del dfanno

len(descriptions_test) = 5000


## At this stage the "descriptions_train_and_val"  dict has info for ALL Coco_Train2017 images = 118287 images. Since, for model training only using 100k images, the extra images info needs to be removed.
### The image encodings contain the 100k image to be retained. So use its keys to retain the correct info in descriptions_train_and_val as they have same key i.e. image file name

In [228]:
print(f"Total entries in descriptions_train_and_val_dataset BEFORE culling = {len(descriptions_train_and_val)}")
retain_dict = {}
for key in img_encodings_train_and_val.keys():
    retain_dict.update({key:descriptions_train_and_val[key]})
descriptions_train_and_val = retain_dict.copy() ## replace with the just identifed 100k images for Train+Validation model training
del retain_dict
print(f"Retained entries in descriptions_train_and_val = {len(descriptions_train_and_val)}")  ## will be for all 100k

Total entries in descriptions_train_and_val_dataset BEFORE culling = 118287
Retained entries in descriptions_train_and_val = 100000


## Data cleaning on the descriptions dicts for descriptions_train_and_val
## First on the descriptions_train_and_val_dataset
##       Later same cleanup on descriptions_test

### Clean up actions done:
### 1. lower casing
### 2. punctuation removal i.e. string.punctuation which coevers '!" HASH DOLLAR_SIGN %&\'()*+,-./:;<=>?@[\\]^_`{|}~' are replaced with a space
### 3. remove any words with length = 1
### 4. remove any words now left as non-alphabetic

In [229]:
## example of caption with a hyphen - in the caption
descriptions_train_and_val['000000005802']

['Two men wearing aprons working in a commercial-style kitchen.',
 'Chefs preparing food in a professional metallic style kitchen.',
 'Two people standing around in a large kitchen.',
 'A commercial kitchen with two men working to prepare several plates.',
 'two men in white shirts in a large steel kitchen']

In [230]:
# prepare translation table for removing punctuation
## string.punctuation  gives   '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' and will take care of all these characters being made into a space
## commented table made "commercial-style kitchen" into "commercialstyle kitchen"
#tran_table = str.maketrans('', '', string.punctuation)
## but this table makes "commercial-style kitchen" into "commercial style kitchen"
tran_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
for key, desc_list in descriptions_train_and_val.items():
    for idx in range(len(desc_list)):
        desc = desc_list[idx]
        # replace all punctuation with space in description before tokenizing
        desc = desc.translate(tran_table)
        # tokenize
        desc = desc.split()
        # convert to lower case
        desc = [word.lower() for word in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove any non-alphabetic tokens
        desc = [word for word in desc if word.isalpha()]
        # overwrite with cleaned description
        desc_list[idx] =  ' '.join(desc)

In [231]:
## example of caption which had the hyphen - in the caption  -- POST CLEANUP
descriptions_train_and_val['000000005802']

['two men wearing aprons working in commercial style kitchen',
 'chefs preparing food in professional metallic style kitchen',
 'two people standing around in large kitchen',
 'commercial kitchen with two men working to prepare several plates',
 'two men in white shirts in large steel kitchen']

## Data cleaning on the descriptions dicts for descriptions_test_dataset

In [232]:
dict(list(descriptions_test.items())[:2])

{'000000179765': ['A black Honda motorcycle parked in front of a garage.',
  'A Honda motorcycle parked in a grass driveway',
  'A black Honda motorcycle with a dark burgundy seat.',
  'Ma motorcycle parked on the gravel in front of a garage',
  'A motorcycle with its brake extended standing outside'],
 '000000190236': ['An office cubicle with four different types of computers.',
  'The home office space seems to be very cluttered.',
  'an office with desk computer and chair and laptop.',
  'Office setting with a lot of computer screens.',
  'A desk and chair in an office cubicle.']}

In [233]:
## example of caption with accidental newline \n in the caption
descriptions_test['000000482917']

['A dog sitting between its masters feet on a footstool watching tv\n',
 'A dog between the feet of a person looking at a TV.',
 'A dog and a person are watching television together.',
 'A person is sitting with their dog watching tv.',
 'A man relaxing at home, watching television with his dog.']

In [234]:
# prepare translation table for removing punctuation
## string.punctuation  gives   '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' and will take care of all these characters being made into a space
tran_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
for key, desc_list in descriptions_test.items():
    for idx in range(len(desc_list)):
        desc = desc_list[idx]
        # replace all punctuation with space in description before tokenizing
        desc = desc.translate(tran_table)
        # tokenize
        desc = desc.split()
        # convert to lower case
        desc = [word.lower() for word in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove any non-alphabetic tokens
        desc = [word for word in desc if word.isalpha()]
        # overwrite with cleaned description
        desc_list[idx] =  ' '.join(desc)

In [235]:
## example of caption with accidental newline \n in the caption  -- POST CLEANUP
descriptions_test['000000482917']

['dog sitting between its masters feet on footstool watching tv',
 'dog between the feet of person looking at tv',
 'dog and person are watching television together',
 'person is sitting with their dog watching tv',
 'man relaxing at home watching television with his dog']

In [236]:
OPDIR

'../working/'

In [237]:
## pickle the cleaned up descriptions_train_and_val    and    descriptions_test

"""
with open( OPDIR + 'descriptions_train_and_val.pkl' , 'wb') as handle:
  pickle.dump(descriptions_train_and_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for descriptions_train_and_val pickled to :: {'descriptions_train_and_val.pkl'} :: len(descriptions_train_and_val) = {len(descriptions_train_and_val)}")


with open( OPDIR + 'descriptions_test.pkl' , 'wb') as handle:
  pickle.dump(descriptions_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for descriptions_test pickled to :: {'descriptions_test.pkl'} :: len(descriptions_test) = {len(descriptions_test)}")
"""

'\nwith open( OPDIR + \'descriptions_train_and_val.pkl\' , \'wb\') as handle:\n  pickle.dump(descriptions_train_and_val, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"Data for descriptions_train_and_val pickled to :: {\'descriptions_train_and_val.pkl\'} :: len(descriptions_train_and_val) = {len(descriptions_train_and_val)}")\n\n\nwith open( OPDIR + \'descriptions_test.pkl\' , \'wb\') as handle:\n  pickle.dump(descriptions_test, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"Data for descriptions_test pickled to :: {\'descriptions_test.pkl\'} :: len(descriptions_test) = {len(descriptions_test)}")\n'

## ALWAYS - create the Train and Validation datasets using the 100k images and pickle the files for reload later
### TRAIN_SIZE = 97k, balance 3k for VALIDATION_SIZE
### Split descriptions data i.e. "descriptions_train_and_val"   into "descriptions_train"  and "descriptions_val"
### Split encodings data    i.e. "img_encodings_train_and_val"  into "img_encodings_train" and "img_encodings_val"

In [238]:
all_train_and_val_keys = [key for key in img_encodings_train_and_val.keys()]
print(f"Number of keys to be used for the split = {len(all_train_and_val_keys)}")

TRAIN_SIZE_N = 97000
train_imgs_keys, val_imgs_keys = train_test_split(all_train_and_val_keys, train_size = TRAIN_SIZE_N, random_state=444)

print(f"Sizes of: Train dataset = {len(train_imgs_keys)},  Validation dataset = {len(val_imgs_keys)}")
del all_train_and_val_keys

Number of keys to be used for the split = 100000
Sizes of: Train dataset = 97000,  Validation dataset = 3000


In [239]:
## use the keys of the just created split files containing the keys to makes the splits for the descriptions and the encodings dicts

descriptions_train = dict()
descriptions_val = dict()
img_encodings_train = dict()
img_encodings_val = dict()

for key in train_imgs_keys:
    descriptions_train.update({key:descriptions_train_and_val.get(key)})
    img_encodings_train.update({key:img_encodings_train_and_val.get(key)})

for key in val_imgs_keys:
    descriptions_val.update({key:descriptions_train_and_val.get(key)})
    img_encodings_val.update({key:img_encodings_train_and_val.get(key)})

print(f"Sizes of: Train description = {len(descriptions_train)},  Validation description = {len(descriptions_val)}")
print(f"Sizes of: Train encodings   = {len(img_encodings_train)}, Validation encodings = {len(img_encodings_val)}")

Sizes of: Train description = 97000,  Validation description = 3000
Sizes of: Train encodings   = 97000, Validation encodings = 3000


In [240]:
OPDIR

'../working/'

In [241]:
## pickle the descriptions , encodings data for train and validation - split is 97k Train 3k for Validation
## pickle also the train_img_keys and val_img_keys 

"""
PICKLE_FILENAME_descriptions_train  = r'descriptions_train_97000.pkl'
PICKLE_FILENAME_descriptions_val    = r'descriptions_val_3000.pkl'
PICKLE_FILENAME_img_encodings_train = r'img_encodings_train_97000.pkl'
PICKLE_FILENAME_img_encodings_val   = r'img_encodings_val_3000.pkl'

with open( OPDIR + PICKLE_FILENAME_descriptions_train , 'wb') as handle:
  pickle.dump(descriptions_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for descriptions_train pickled to :: {PICKLE_FILENAME_descriptions_train}")

with open( OPDIR + PICKLE_FILENAME_descriptions_val , 'wb') as handle:
  pickle.dump(descriptions_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for descriptions_val pickled to :: {PICKLE_FILENAME_descriptions_val}")

with open( OPDIR + PICKLE_FILENAME_img_encodings_train , 'wb') as handle:
  pickle.dump(img_encodings_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for img_encodings_train pickled to :: {PICKLE_FILENAME_img_encodings_train}")

with open( OPDIR + PICKLE_FILENAME_img_encodings_val , 'wb') as handle:
  pickle.dump(img_encodings_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for img_encodings_val pickled to :: {PICKLE_FILENAME_img_encodings_val}")


PICKLE_FILENAME_train_imgs_keys   = r'train_imgs_keys_97000_randomSplit444.pkl'
PICKLE_FILENAME_val_imgs_keys     = r'val_imgs_keys_3000_randomSplit444.pkl'

with open( OPDIR + PICKLE_FILENAME_train_imgs_keys , 'wb') as handle:
  pickle.dump(train_imgs_keys, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for train_imgs_keys pickled to :: {PICKLE_FILENAME_train_imgs_keys}")

with open( OPDIR + PICKLE_FILENAME_val_imgs_keys , 'wb') as handle:
  pickle.dump(val_imgs_keys, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for val_imgs_keys pickled to :: {PICKLE_FILENAME_val_imgs_keys}")
"""

'\nPICKLE_FILENAME_descriptions_train  = r\'descriptions_train_97000.pkl\'\nPICKLE_FILENAME_descriptions_val    = r\'descriptions_val_3000.pkl\'\nPICKLE_FILENAME_img_encodings_train = r\'img_encodings_train_97000.pkl\'\nPICKLE_FILENAME_img_encodings_val   = r\'img_encodings_val_3000.pkl\'\n\nwith open( OPDIR + PICKLE_FILENAME_descriptions_train , \'wb\') as handle:\n  pickle.dump(descriptions_train, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"Data for descriptions_train pickled to :: {PICKLE_FILENAME_descriptions_train}")\n\nwith open( OPDIR + PICKLE_FILENAME_descriptions_val , \'wb\') as handle:\n  pickle.dump(descriptions_val, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"Data for descriptions_val pickled to :: {PICKLE_FILENAME_descriptions_val}")\n\nwith open( OPDIR + PICKLE_FILENAME_img_encodings_train , \'wb\') as handle:\n  pickle.dump(img_encodings_train, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"Data for img_encodings_train pickled to :: {PICKLE_FILENAME_im

In [242]:
os.listdir(OPDIR)

['weights_out', '__notebook_source__.ipynb']

## ALWAYS - Add the    "startseq"    and    "endseq"    tokens to all the training example descriptions

In [243]:
## see the unchanged descriptions - note no startseq and endseq present now
i=0
for k, v in descriptions_train.items():
  i+=1
  if i>5:
    break
  print(k, v)

000000044704 ['jar of liquid next to blender full of liquid', 'jar full of red blended good next to blender filled with the same', 'red drink that was made in blender', 'blender and clear glass next to each other with red liquid in them', 'blender and jar of red liquid on table']
000000357865 ['bathroom with toilet and sink with cup on it', 'white toilet sitting next to white sink in bathroom', 'small bathroom has towels rugs and gray floor', 'an image of bathroom setting with toilet and counter', 'bathroom with toilet sink trash can towel rack and mirror']
000000131172 ['group of people are sitting on bench outside', 'man sitting by woman and girl who are on their cell phones', 'black and white photograph of people on bench table', 'lot of people are sitting on the bench', 'man and his daughters sit on bench']
000000466024 ['two bears play in water beside cement wall', 'two bears wrestle in pool of water', 'two bears are holding on to each other in the middle of water', 'two bears hol

In [244]:
## add startseq and endseq to the captions of all entries in descriptions_train

for k, v in descriptions_train.items():
    updated_values = [''.join(['startseq ', each_desc, ' endseq']) for each_desc in v]
    descriptions_train.update( { k : updated_values } )

In [245]:
## see the updated descriptions - note startseq and endseq IS PRESENT at both ends for all the descriptions
i=0
for k, v in descriptions_train.items():
  i+=1
  if i>5:
    break
  print(k, v)

000000044704 ['startseq jar of liquid next to blender full of liquid endseq', 'startseq jar full of red blended good next to blender filled with the same endseq', 'startseq red drink that was made in blender endseq', 'startseq blender and clear glass next to each other with red liquid in them endseq', 'startseq blender and jar of red liquid on table endseq']
000000357865 ['startseq bathroom with toilet and sink with cup on it endseq', 'startseq white toilet sitting next to white sink in bathroom endseq', 'startseq small bathroom has towels rugs and gray floor endseq', 'startseq an image of bathroom setting with toilet and counter endseq', 'startseq bathroom with toilet sink trash can towel rack and mirror endseq']
000000131172 ['startseq group of people are sitting on bench outside endseq', 'startseq man sitting by woman and girl who are on their cell phones endseq', 'startseq black and white photograph of people on bench table endseq', 'startseq lot of people are sitting on the bench 

In [246]:
## So now following data is created:

### Image ecnodings from Inception-v3 pretrained on Imagenet
### Train      data = 97000 images = img_encodings_train variable
### Validation data = 3000  images = img_encodings_val   variable
### Test       data = 5000  images = img_encodings_test  variable

### Image names with their descriptions - after cleaning up
### Train      data = 97000 images = descriptions_train variable : only this has the startseq and endseq tokens inserted at either end
### Validation data = 3000  images = descriptions_val   variable
### Test       data = 5000  images = descriptions_test  variable

In [247]:
print(f"Encodings data:\nlen(img_encodings_train) = {len(img_encodings_train)}\tlen(img_encodings_val) = {len(img_encodings_val)}\tlen(img_encodings_test) = {len(img_encodings_test)}")
print(f"Descriptions data:\nlen(descriptions_train) = {len(descriptions_train)}\tlen(descriptions_val) = {len(descriptions_val)}\tlen(descriptions_test) = {len(descriptions_test)}")

Encodings data:
len(img_encodings_train) = 97000	len(img_encodings_val) = 3000	len(img_encodings_test) = 5000
Descriptions data:
len(descriptions_train) = 97000	len(descriptions_val) = 3000	len(descriptions_test) = 5000


In [248]:
## pickle the descriptions_train which now has the start and end sequences inserted
"""
PICKLE_FILENAME_descriptions_train_startend  = r'descriptions_train_97000_startend.pkl'

with open( OPDIR + PICKLE_FILENAME_descriptions_train_startend , 'wb') as handle:
  pickle.dump(descriptions_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Data for descriptions_train with start end seq pickled to :: {PICKLE_FILENAME_descriptions_train_startend}")
"""

'\nPICKLE_FILENAME_descriptions_train_startend  = r\'descriptions_train_97000_startend.pkl\'\n\nwith open( OPDIR + PICKLE_FILENAME_descriptions_train_startend , \'wb\') as handle:\n  pickle.dump(descriptions_train, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"Data for descriptions_train with start end seq pickled to :: {PICKLE_FILENAME_descriptions_train_startend}")\n'

## ALWAYS - Vocabulary building
## Create the vocabulary based on the captions in the training descriptions with the start and end sequence tokens added at either end

### First for all the words i.e. variable "vocabulary"
### Then culled based on frequency > threshold i.e. variable "vocab_threshold"
#### Later used the size of the wordtoix list (built using vocab_threshold), added 1 to make that equal to VOCAB_LEN for the model. The additional of one is to account for the 0 padding value.

In [249]:
## at this stage the descriptions_train already has the start and end tokens added to it
vocabulary = set()
for key in descriptions_train.keys():
    [vocabulary.update(d.split()) for d in descriptions_train[key]]
print(f"Original Vocabulary Size with all words = {len(vocabulary)}")

Original Vocabulary Size with all words = 24299


In [250]:
## This means we have some large number of unique words in the corpus - across all the descriptions for the 97000 train set images and including the start and end sequence words
## 
## However, if we think about it, many of these words will occur very few times, 
## say 1, 2 or 3 times. Since we are creating a predictive model, we would not like 
## to have all the words present in our vocabulary but the words which are more 
## likely to occur or which are common. This helps the model become more robust 
## to outliers and make less mistakes.
##
## Hence we consider only those words which more often in the corpus than some chosen threshold value.

### Cull vocabulary based on frequency > threshold
### Capturing new data into variable   vocab_threshold

In [251]:
# Create a list of all the training captions, find the freq and retain words where the freq > threshold chosen

all_desc_in_training_samples = []
for key, val in descriptions_train.items():
    for cap in val:
        all_desc_in_training_samples.append(cap)

MIN_WORD_COUNT_THRESHOLD = 10
word_counts = {}
nsents = 0
for each_desc in all_desc_in_training_samples:
    nsents += 1
    for w in each_desc.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab_threshold = [w for w in word_counts if word_counts[w] >= MIN_WORD_COUNT_THRESHOLD]

print(f"Culled vocabulary to only retain words occurring more than threshold = {MIN_WORD_COUNT_THRESHOLD} times.\nNew vocab size , len(vocab_threshold) = {len(vocab_threshold)}")

Culled vocabulary to only retain words occurring more than threshold = 10 times.
New vocab size , len(vocab_threshold) = 6759


In [252]:
## Thus from the large number of unique words in original vocabulary,
## the count of unique words in culled vocab vocab_threshold is substantially reduced.
## This reduces the set of possible words for Decoder less, so model is robust, trains wells.
## 

## So we now have two vocabulary variables:
#### 1) vocabulary
####    list of ALL unique words in our 97k training captions
#### 2) vocab_threshold
####    list of unique words occuring MORE than the threshold limit

## NOTE: Eventually though we will be zero-padding, so the total words = len(vocab_threshold) + 1
##       Additional one index for the 0-pad value

## ALWAYS - Data Preprocessing — Captions
### 1) Build the word and index mapping dictionaries based on the reduced vocabulary.
### 2) Find the maximum length of caption description
###    NOTE: This is using the train images descriptions WITH the startseq and endseq word insertions

In [253]:
### We want to predict the captions. So while training, the captions are the target variables (Y)
### But the prediction of caption happens word by word, not in one go.
### So encode each word into a fixed sized vector later (GloVe).
### Here create the word-to-index and index-to-word dictionaries.
###
### We represent every unique word in the vocabulary by an integer (index).
### From the culled vocab (high freq words including the zero), we have 6760 unique words
###      in our corpus and thus each word will be represented by an integer index between 1 to 6760.
###
### Create two dictionaries: using the final vocabulary that model should output including the start and end sequence special tokens
###        wordtoix[‘walking’] -> returns index of the word ‘walking’ e.g. it is 32
###        ixtoword[i] -> returns the word whose index is ‘i’ e.g. ixtoword[32] will be 'walking'
### Later, while building the RNN-Decoder model, its parameter VOCAB_SIZE is calculated as = len(wordtoix) + 1

In [254]:
## NOTE: using the vocab_threhold
ixtoword = dict()
wordtoix = dict()
idx = 1 ## index 1 is starting value as the 0 will be mapped to the 0-pad value
for w in vocab_threshold:
    wordtoix[w] = idx
    ixtoword[idx] = w
    idx += 1

### As wordtoix and ixtoword lists are required for the inference later - pickle them too

In [255]:
"""
PICKLE_FILENAME_wordtoix  = r'wordtoix_train_97000.pkl'
PICKLE_FILENAME_ixtoword  = r'ixtoword_train_97000.pkl'

with open( OPDIR + PICKLE_FILENAME_wordtoix , 'wb') as handle:
  pickle.dump(wordtoix, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"wordtoix pickled to :: {PICKLE_FILENAME_wordtoix}")

with open( OPDIR + PICKLE_FILENAME_ixtoword , 'wb') as handle:
  pickle.dump(ixtoword, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"ixtoword pickled to :: {PICKLE_FILENAME_ixtoword}")
"""

'\nPICKLE_FILENAME_wordtoix  = r\'wordtoix_train_97000.pkl\'\nPICKLE_FILENAME_ixtoword  = r\'ixtoword_train_97000.pkl\'\n\nwith open( OPDIR + PICKLE_FILENAME_wordtoix , \'wb\') as handle:\n  pickle.dump(wordtoix, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"wordtoix pickled to :: {PICKLE_FILENAME_wordtoix}")\n\nwith open( OPDIR + PICKLE_FILENAME_ixtoword , \'wb\') as handle:\n  pickle.dump(ixtoword, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"ixtoword pickled to :: {PICKLE_FILENAME_ixtoword}")\n'

In [256]:
dict(list(descriptions_train.items())[:2])

{'000000044704': ['startseq jar of liquid next to blender full of liquid endseq',
  'startseq jar full of red blended good next to blender filled with the same endseq',
  'startseq red drink that was made in blender endseq',
  'startseq blender and clear glass next to each other with red liquid in them endseq',
  'startseq blender and jar of red liquid on table endseq'],
 '000000357865': ['startseq bathroom with toilet and sink with cup on it endseq',
  'startseq white toilet sitting next to white sink in bathroom endseq',
  'startseq small bathroom has towels rugs and gray floor endseq',
  'startseq an image of bathroom setting with toilet and counter endseq',
  'startseq bathroom with toilet sink trash can towel rack and mirror endseq']}

In [257]:
## determine the maximum sequence length - parameter MAX_LENGTH_CAPTION used during the RNN deocder model setup

## convert a dictionary of clean descriptions to a list of descriptions
def extract_each_desc(_descriptions):
    all_desc = list()
    for key in _descriptions.keys():
        [all_desc.append(d) for d in _descriptions[key]]
    return all_desc

## find the longest description length
def find_max_length_desc(_descriptions):
    desc_sentences = extract_each_desc(_descriptions)
    return max(len(d.split()) for d in desc_sentences)

MAX_LENGTH_CAPTION = find_max_length_desc(descriptions_train)  ## will be used directly later while defining Decoder model
print(f"Max Description Length: {MAX_LENGTH_CAPTION}")

Max Description Length: 49


### Checking what sentences had the max length and get an idea of how many sentences had what lengths

In [258]:
[cap for val in descriptions_train.values() for cap in val][:8]

['startseq jar of liquid next to blender full of liquid endseq',
 'startseq jar full of red blended good next to blender filled with the same endseq',
 'startseq red drink that was made in blender endseq',
 'startseq blender and clear glass next to each other with red liquid in them endseq',
 'startseq blender and jar of red liquid on table endseq',
 'startseq bathroom with toilet and sink with cup on it endseq',
 'startseq white toilet sitting next to white sink in bathroom endseq',
 'startseq small bathroom has towels rugs and gray floor endseq']

In [259]:
df = pd.DataFrame(data=[cap for val in descriptions_train.values() for cap in val], columns=['sent'])
df['caplen'] = df['sent'].str.split().apply(len)
df['sent'][df['caplen'] == MAX_LENGTH_CAPTION]

101875    startseq two men holding tennis rackets up in ...
Name: sent, dtype: object

In [260]:
df = pd.DataFrame(data=[cap for val in descriptions_train.values() for cap in val], columns=['sent'])
count = df['sent'].str.split().apply(len).value_counts()
count.index = count.index.astype(str)
count.sort_index(inplace=True, ascending=False)
count.head(15)

9     94555
8     37858
7      3533
6         4
5         1
49        1
46        4
45        1
44        7
43        9
42       13
41       15
40       14
4         2
39       16
Name: sent, dtype: int64

In [261]:
del df

## ALWAYS - RELOAD FROM PICKLE FILE:  Reload the GloVe vectors related data structures

### Sizes of reloaded pickled data should be =
#### Size of embeddings_dict_glove200 = 20.971616 MB

### Citing GloVe: Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]GloVe: Global Vectors for Word Representation. [pdf] [bib]
### Link: More details on this here- https://nlp.stanford.edu/projects/glove/

In [262]:
## Already downloaded the glove file and used it to create the embeddings dict and pickled it. Reloading now.
###    This data is made available under the Public Domain Dedication and License v1.0 whose full text can be found at: http://www.opendatacommons.org/licenses/pddl/1.0/.
###    Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, 822 MB download): glove.6B.zip
###    Common Crawl (42B tokens, 1.9M vocab, uncased, 300d vectors, 1.75 GB download): glove.42B.300d.zip
###    Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download): glove.840B.300d.zip
###    Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 25d, 50d, 100d, & 200d vectors, 1.42 GB download): glove.twitter.27B.zip

### Am using glove.6B.zip on 14.09.2020
### Then its glove.6B.200d.txt file for the 200 dimensional word embeddings

In [263]:
IPDIR_EMBED_DICT

'../input/embeddings-glove200/'

In [264]:
## reload from picked file


with open( IPDIR_EMBED_DICT + 'glove200_embeddings_dict_1.pkl', 'rb') as handle:
    embeddings_dict_glove200 = pickle.load(handle)

print(f"Number of words in this embeddings file = {len(embeddings_dict_glove200)}")
print(f"Shape of random word = {embeddings_dict_glove200['1917'].shape}")
print(f"Shape of another random word = {embeddings_dict_glove200['sculpture'].shape}")
print(f"Size of embeddings_dict_glove200 = {sys.getsizeof(embeddings_dict_glove200)/1000000} MB")

Number of words in this embeddings file = 400000
Shape of random word = (200,)
Shape of another random word = (200,)
Size of embeddings_dict_glove200 = 20.971624 MB


### Now create the EMBEDDINGS MATRIX using the embeddings dict

In [265]:
## Now, for all the unique words the model should possibly output as its vocabulary, we create an embedding matrix

In [266]:
## the value now, as it will be used as:: VOCAB_SIZE = len(wordtoix) + 1
len(wordtoix)

6759

In [267]:
## see the index output by wordtoix for the start and end sequence tokens as well as some random one word
print( wordtoix.get('startseq') , wordtoix.get('endseq') , wordtoix.get('cat') )

1 9 247


In [268]:
## since using GloVe-200, each word is represented as a 200 dimensional vector
EMBEDDING_DIMS = 200
## VOCAB_SIZE = the parameter to be used while defining the decoder model
VOCAB_SIZE = len(wordtoix) + 1 # number of unique high freq words in the vocabulary + 1 (one index for the 0)

embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIMS))
for word, i in wordtoix.items():
    #if i < max_words:
    ## using .get(word) method and not [word] as second method throws key error, while first returns none.
    ##        E.g.    print(embeddings_dict_glove200.get('sculture')) = None
    ##            But print(embeddings_dict_glove200['sculture'}) will break with key error
    embedding_vector = embeddings_dict_glove200.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

print(f"Shape of embedding_matrix = {embedding_matrix.shape}")

Shape of embedding_matrix = (6760, 200)


In [269]:
## e.g. of a word PRESENT the GoVe200 data file
mytestword = 'sculpture'
print(f"Index position of word {mytestword} in the vocabulary = {wordtoix.get(mytestword)}")
print(f"\nEmbedding vector for word 'sculpture' from GloVe directly = {embeddings_dict_glove200.get(mytestword)}")
print(f"\nembedding_matrix entry for '{mytestword}' = {embeddings_dict_glove200.get(mytestword)}")
## it should match

Index position of word sculpture in the vocabulary = 2027

Embedding vector for word 'sculpture' from GloVe directly = [ 0.24496   -0.43193   -0.45643   -0.19918    0.51172   -1.0377
  0.25122    0.12462    0.10547   -0.97614    0.53084   -0.1065
  0.37079    0.68693    0.87499   -0.20523   -0.3771     0.2687
  0.83217   -0.31284    0.014147   1.2787     0.68525   -0.11444
  0.16005    0.72932    0.39781   -0.69046   -0.04684   -0.16001
  0.16502   -0.25699    0.13922   -0.12093   -0.048757   0.0030296
 -0.51352   -0.79619    0.78581   -0.42436   -0.47878   -0.28729
  0.36649    0.27928    0.32741    0.37925    0.59473   -0.28681
  0.066106   0.29147    0.059296   0.71001   -0.14208   -0.063589
  0.076285  -0.027859   0.092648   0.16012    0.36895   -0.37574
 -0.27742    0.2595     0.060266   0.0081959 -0.4189     0.46089
 -0.65505   -0.13595   -0.47514   -0.5632    -0.2457    -0.85826
 -0.25973    0.66389    0.20883   -0.34182   -0.29798    0.10557
 -0.87879   -0.51638    0.7632    -0

In [270]:
## e.g. of a word NOT PRESENT the GoVe200 data file
## answer None in all cases since the word is not in the vocabulary itself the other two will throw key error
mytestword = 'sculture'
print(f"Index position of word {mytestword} in the vocabulary = {wordtoix.get(mytestword)}")
print(f"\nEmbedding vector for word 'sculpture' from GloVe directly = {embeddings_dict_glove200.get(mytestword)}")
print(f"\nembedding_matrix entry for '{mytestword}' = {embeddings_dict_glove200.get(mytestword)}")

Index position of word sculture in the vocabulary = None

Embedding vector for word 'sculpture' from GloVe directly = None

embedding_matrix entry for 'sculture' = None


In [271]:
del embeddings_dict_glove200

In [272]:
embedding_matrix.shape

(6760, 200)

In [273]:
## pickle the embedding matrix just created
"""
PICKLE_FILENAME_embedding_matrix  = r'embedding_matrix_6758_200.pkl'

with open( OPDIR + PICKLE_FILENAME_embedding_matrix , 'wb') as handle:
  pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"embedding_matrix with shape {embedding_matrix.shape} pickled to :: {PICKLE_FILENAME_embedding_matrix}")
"""

'\nPICKLE_FILENAME_embedding_matrix  = r\'embedding_matrix_6758_200.pkl\'\n\nwith open( OPDIR + PICKLE_FILENAME_embedding_matrix , \'wb\') as handle:\n  pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint(f"embedding_matrix with shape {embedding_matrix.shape} pickled to :: {PICKLE_FILENAME_embedding_matrix}")\n'

## Data Preparation using Generator Function

In [274]:
## Hereafter, I will try to explain the remaining steps by taking a sample example as follows:
## Consider we have 3 images and their 3 corresponding captions as follows:
## 
## (Train image 1) Caption -> The black cat sat on grass
## (Train image 2) Caption -> The white cat is walking on road
## (Test image) Caption -> The black cat is walking on grass
## 
## Now, let’s say we use the first two images and their captions to train the model and the third image to test our model.
## Now the questions that will be answered are: how do we frame this as a supervised learning problem?, what does the data matrix look like? how many data points do we have?, etc.
## First we need to convert both the images to their corresponding 2048 length feature vector as discussed above. Let “Image_1” and “Image_2” be the feature vectors of the first two images respectively
## Secondly, let’s build the vocabulary for the first two (train) captions by adding the two tokens “startseq” and “endseq” in both of them: (Assume we have already performed the basic cleaning steps)
## 
## Caption_1 -> “startseq the black cat sat on grass endseq”
## Caption_2 -> “startseq the white cat is walking on road endseq”
## 
## vocab = {black, cat, endseq, grass, is, on, road, sat, startseq, the, walking, white}
## 
## Let’s give an index to each word in the vocabulary:
## black -1, cat -2, endseq -3, grass -4, is -5, on -6, road -7, sat -8, startseq -9, the -10, walking -11, white -12
## 
## Now let’s try to frame it as a supervised learning problem where we have a set of data points D = {Xi, Yi}, where Xi is the feature vector of data point ‘i’ and Yi is the corresponding target variable.
## 
## Let’s take the first image vector Image_1 and its corresponding caption “startseq the black cat sat on grass endseq”. Recall that, Image vector is the input and the caption is what we need to predict. But the way we predict the caption is as follows:
## For the first time, we provide the image vector and the first word as input and try to predict the second word, i.e.:
## Input = Image_1 + ‘startseq’; Output = ‘the’
## Then we provide image vector and the first two words as input and try to predict the third word, i.e.:
## Input = Image_1 + ‘startseq the’; Output = ‘cat’
## And so on . . .
## 
## Thus, we can summarize the data matrix for one image and its corresponding caption as follows:
## Step 1 :: image feature = FeatVecImg1 :: Partial Caption = "startseq" :: Target Word = "the"
## Step 2 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the" :: Target Word = "black"
## Step 3 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black" :: Target Word = "cat"
## Step 4 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat" :: Target Word = "sat"
## Step 5 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat sat" :: Target Word = "on"
## Step 6 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat sat on" :: Target Word = "grass"
## Step 7 :: image feature = FeatVecImg1 :: Partial Caption = "startseq the black cat sat on grass" :: Target Word = "endseq"
## 
## It must be noted that, one image+caption is not a single data point but are multiple data points depending on the length of the caption.
## All the above 7 data points together constitute the full data for one image and its caption!!!!
## Similarly for second images, there will be multiple steps together that consitute its full data point.
## 
## We must now understand that in every data point, it’s not just the image which goes as input to the system, but also, a partial caption which helps to predict the next word in the sequence.
## Since we are processing sequences, we will employ a Recurrent Neural Network to read these partial captions (more on this later).
## However, we have already discussed that we are not going to pass the actual English text of the caption, rather we are going to pass the sequence of indices where each index represents a unique word.
## 
## Since we have already created an index for each word, let’s now replace the words with their indices and understand how the data matrix will look like:
## Step 1  :: image feature = FeatVecImg1 :: Partial Caption = "9" :: Target Word = "10"
## Step 2  :: image feature = FeatVecImg1 :: Partial Caption = "9 10" :: Target Word = "1"
## Step 3  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1" :: Target Word = "2"
## Step 4  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2" :: Target Word = "8"
## Step 5  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2 8" :: Target Word = "6"
## Step 6  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2 8 6" :: Target Word = "4"
## Step 7  :: image feature = FeatVecImg1 :: Partial Caption = "9 10 1 2 8 6 4" :: Target Word = "3"
## Step 8  :: image feature = FeatVecImg2 :: Partial Caption = "9" :: Target Word = "10"
## Step 9  :: image feature = FeatVecImg2 :: Partial Caption = "9 10" :: Target Word = "12"
## Step 10 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12" :: Target Word = "2"
## Step 11 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2" :: Target Word = "5"
## Step 12 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5" :: Target Word = "11"
## Step 13 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5 11" :: Target Word = "6"
## Step 14 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5 11 6" :: Target Word = "7"
## Step 15 :: image feature = FeatVecImg2 :: Partial Caption = "9 10 12 2 5 11 6 3" :: Target Word = "3"
## 
## Since we would be doing batch processing (explained later), we need to make sure that each sequence is of equal length. Hence we need to append 0’s (zero padding) at the end of each sequence. But how many zeros should we append in each sequence?
## Well, this is the reason we had calculated the maximum length of a caption. So we will append those many number of zeros which will lead to every sequence having a length = maximum length of caption.
## 
## Step 1  :: image feature = FeatVecImg1 :: Partial Caption = "[9 0 0 ...]" :: Target Word = "10"
## Step 2  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 0 0 ...]" :: Target Word = "1"
## Step 3  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 0 0 ...]" :: Target Word = "2"
## Step 4  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 0 0 ...]" :: Target Word = "8"
## Step 5  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 8 0 0 ...]" :: Target Word = "6"
## Step 6  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 8 6 0 0 ...]" :: Target Word = "4"
## Step 7  :: image feature = FeatVecImg1 :: Partial Caption = "[9 10 1 2 8 6 4 0 0 ...]" :: Target Word = "3"
## Step 8  :: image feature = FeatVecImg2 :: Partial Caption = "[9 0 0 ...]" :: Target Word = "10"
## Step 9  :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 0 0 ...]" :: Target Word = "12"
## Step 10 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 0 0 ...]" :: Target Word = "2"
## Step 11 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 0 0 ...]" :: Target Word = "5"
## Step 12 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 0 0 ...]" :: Target Word = "11"
## Step 13 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 11 0 0 ...]" :: Target Word = "6"
## Step 14 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 11 6 0 0 ...]" :: Target Word = "7"
## Step 15 :: image feature = FeatVecImg2 :: Partial Caption = "[9 10 12 2 5 11 6 3 0 0 ...]" :: Target Word = "3"
## Appended adequete 0's to each partial caption to make its length = max length
## 
## 
## 
## Need for a Data Generator:
## In the above example, I have only considered 2 images and captions which have lead to 15 data points.
## However, in our actual training dataset we have 6000 images, each having 5 captions. This makes a total of 30000 images and captions.
## Even if we assume that each caption on an average is just 7 words long, it will lead to a total of 30000*7 i.e. 210000 data points.
## 
## Compute the size of the data matrix:
## 
## 
##    img_vector = 2048        partial caption (length = max caption length)
## ---------------------------------------------------------------------------
## -                       -                                                 -
## -                       -                                                 -
## -                       -                                                 -
## -                       -                                                 -
## ---------------------------------------------------------------------------
## Say above matrix has n rows, m columns, so Size of the data matrix = n*m
## n-> number of data points (assumed as 210000)  (6000 images * 5 captions per image * 7 words average legnth of each caption)
## m-> length of each data point
##     = 2048 + length of partial caption (say something)
##     = 2048 + something
## 
## Now this "something"  NOTE EQUAL to the max length of caption!
## Every word (or index) will be mapped (embedded) to higher dimensional space through one of the word embedding techniques.
## As we using GloVe-200, each embedding has 200 floats representing each number.
## 
## So with each caption sentence consisting of max length caption size word-indexes, each word represented by 200 dimensional value:
##    Assuming max lenght of caption = 45
##    means "something" = 2048 + ( 45 * 200 ) = 2048 + 9000 = 11048 float values to represent each sentence
## 
## Therefore, size of data matrix = m*n = 210000 * 11048 = xxx float values!!!
## Assuming float takes 2 bytes (very conservative), that still means xxx * 2 = xxx GB
## 
## This is pretty huge requirement and even if we are able to manage to load this much data into the RAM, it will make the system very slow.
## For this reason we use data generators a lot in Deep Learning. Data Generators are a functionality which is natively implemented in Python. The ImageDataGenerator class provided by the Keras API is nothing but an implementation of generator function in Python.
## 
## So how does using a generator function solve this problem?
## If you know the basics of Deep Learning, then you must know that to train a model on a particular dataset, we use some version of Stochastic Gradient Descent (SGD) like Adam, Rmsprop, Adagrad, etc.
## With SGD, we do not calculate the loss on the entire data set to update the gradients. Rather in every iteration, we calculate the loss on a batch of data points (typically 64, 128, 256, etc.) to update the gradients.
## 
## This means that we do not require to store the entire dataset in the memory at once. Even if we have the current batch of points in the memory, it is sufficient for our purpose.
## A generator function in Python is used exactly for this purpose. It’s like an iterator which resumes the functionality from the point it left the last time it was called.
## To understand more about Generators, please read here (https://wiki.python.org/moin/Generators).
## 

In [275]:
# data generator, use during the call to model.fit_generator() to create batchwise data
def data_generator_1(_descriptions, _imgs_features_arr, _wordtoix, _max_length, _images_per_batch, _vocab_size):
    X1, X2, y = [] , [] , []  ## empty lists to populate the input and target data for a bath
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in _descriptions.items():
            n+=1
            # retrieve the encoded features of image
            img_feat = _imgs_features_arr[ key ] ## keys in the image encodings dict and descriptions dict use image_filename without the .jpg
            for desc in desc_list:
                # encode the sequence
                seq = [_wordtoix[word] for word in desc.split(' ') if word in _wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = keras.preprocessing.sequence.pad_sequences([in_seq], maxlen=_max_length)[0]
                    # encode output sequence
                    out_seq = keras.utils.to_categorical([out_seq], num_classes=_vocab_size)[0]
                    # store
                    X1.append(img_feat)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n == _images_per_batch:
                ## ValueError: No gradients provided for any variable: ['dense_2/kernel:0', 'dense_2/bias:0', 'lstm_1/lstm_cell_1/kernel:0', 'lstm_1/lstm_cell_1/recurrent_kernel:0', 'lstm_1/lstm_cell_1/bias:0', 'dense_3/kernel:0', 'dense_3/bias:0', 'dense_4/kernel:0', 'dense_4/bias:0'].
                #yield [[np.array(X1), np.array(X2)], np.array(y)]
                yield [np.array(X1), np.array(X2)], np.array(y)
                X1, X2, y = list(), list(), list()
                n=0

# Already ran the above steps earlier and pickled all the variables created based on a certain split of Train and Validation datasets 97k:3k.
## Overwriting all the variables by reloading from the saved pickled files.
## Run necessary code again to check all good to proceed with Training phase:
## Outcome expected:
## 1) unique words in original vocabulary = 24323
## 2) unique words in culled vocab vocab_threshold = 6757
##    Thus, VOCAB_SIZE = 6757 + 1 = 6758
## 3) Embedding matrix shape should be (6758,200)

In [276]:
IPDIR_DETERMINISTIC

'../input/thesis-imgcap-run2-deterministic-info/'

In [277]:
os.listdir(IPDIR_DETERMINISTIC)

['embedding_matrix_6758_200.pkl',
 'val_imgs_keys_3000_randomSplit444.pkl',
 'ixtoword_train_97000.pkl',
 'img_encodings_val_3000.pkl',
 'descriptions_train_97000_startend.pkl',
 'train_imgs_keys_97000_randomSplit444.pkl',
 'descriptions_val_3000.pkl',
 'descriptions_train_and_val.pkl',
 'descriptions_train_97000.pkl',
 'descriptions_test.pkl',
 'img_encodings_train_97000.pkl',
 'wordtoix_train_97000.pkl']

In [278]:
## Reload all the files to make it deterministic


# variable = img_encodings_train
with open(IPDIR_DETERMINISTIC + 'img_encodings_train_97000.pkl', 'rb') as handle:
    img_encodings_train = pickle.load(handle)

# variable = img_encodings_val
with open(IPDIR_DETERMINISTIC + 'img_encodings_val_3000.pkl', 'rb') as handle:
    img_encodings_val = pickle.load(handle)

# variable = descriptions_train
with open(IPDIR_DETERMINISTIC + 'descriptions_train_97000_startend.pkl', 'rb') as handle:
    descriptions_train = pickle.load(handle)

# variable = descriptions_val
with open(IPDIR_DETERMINISTIC + 'descriptions_val_3000.pkl', 'rb') as handle:
    descriptions_val = pickle.load(handle)

# variable = train_imgs_keys
with open(IPDIR_DETERMINISTIC + 'train_imgs_keys_97000_randomSplit444.pkl', 'rb') as handle:
    train_imgs_keys = pickle.load(handle)

# variable = val_imgs_keys
with open(IPDIR_DETERMINISTIC + 'val_imgs_keys_3000_randomSplit444.pkl', 'rb') as handle:
    val_imgs_keys = pickle.load(handle)

# variable = embedding_matrix
with open(IPDIR_DETERMINISTIC + 'embedding_matrix_6758_200.pkl', 'rb') as handle:
    embedding_matrix = pickle.load(handle)

# variable = ixtoword
with open(IPDIR_DETERMINISTIC + 'ixtoword_train_97000.pkl', 'rb') as handle:
    ixtoword = pickle.load(handle)

# variable = wordtoix
with open(IPDIR_DETERMINISTIC + 'wordtoix_train_97000.pkl', 'rb') as handle:
    wordtoix = pickle.load(handle)


In [279]:
## check the reloaded files

In [281]:
print(f"Encodings data:\nlen(img_encodings_train) = {len(img_encodings_train)}\t\tlen(img_encodings_val) = {len(img_encodings_val)}")
print(f"Descriptions data:\nlen(descriptions_train) = {len(descriptions_train)}\t\tlen(descriptions_val) = {len(descriptions_val)}")
print(f"\nCHECK : reloaded values = 97k for Train , 3k for Validation")

Encodings data:
len(img_encodings_train) = 97000		len(img_encodings_val) = 3000
Descriptions data:
len(descriptions_train) = 97000		len(descriptions_val) = 3000

CHECK : reloaded values = 97k for Train , 3k for Validation


In [282]:
## at this stage the descriptions_train already has the start and end tokens added to it
vocabulary = set()
for key in descriptions_train.keys():
    [vocabulary.update(d.split()) for d in descriptions_train[key]]
print(f"Original Vocabulary Size with all words = {len(vocabulary)}")
print(f"\nCHECK : reloaded value = 24323")

Original Vocabulary Size with all words = 24323

CHECK : reloaded value = 24323


In [283]:
# Create a list of all the training captions, find the freq and retain words where the freq > threshold chosen

all_desc_in_training_samples = []
for key, val in descriptions_train.items():
    for cap in val:
        all_desc_in_training_samples.append(cap)

MIN_WORD_COUNT_THRESHOLD = 10
word_counts = {}
nsents = 0
for each_desc in all_desc_in_training_samples:
    nsents += 1
    for w in each_desc.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab_threshold = [w for w in word_counts if word_counts[w] >= MIN_WORD_COUNT_THRESHOLD]

print(f"Culled vocabulary to only retain words occurring more than threshold = {MIN_WORD_COUNT_THRESHOLD} times.\nNew vocab size , len(vocab_threshold) = {len(vocab_threshold)}")
print(f"\nCHECK : reloaded value = 6757")

Culled vocabulary to only retain words occurring more than threshold = 10 times.
New vocab size , len(vocab_threshold) = 6757

CHECK : reloaded value = 6757


In [284]:
## determine the maximum sequence length - parameter MAX_LENGTH_CAPTION used during the RNN deocder model setup

## convert a dictionary of clean descriptions to a list of descriptions
def extract_each_desc(_descriptions):
    all_desc = list()
    for key in _descriptions.keys():
        [all_desc.append(d) for d in _descriptions[key]]
    return all_desc

## find the longest description length
def find_max_length_desc(_descriptions):
    desc_sentences = extract_each_desc(_descriptions)
    return max(len(d.split()) for d in desc_sentences)

MAX_LENGTH_CAPTION = find_max_length_desc(descriptions_train)  ## will be used directly later while defining Decoder model
print(f"Max Description Length: {MAX_LENGTH_CAPTION}")
print(f"\nCHECK : reloaded value = 49")

Max Description Length: 49

CHECK : reloaded value = 49


In [285]:
## the value now, as it will be used as:: VOCAB_SIZE = len(wordtoix) + 1
print(f"len(wordtoix) = {len(wordtoix)}")
print(f"\nCHECK : reloaded value = 6757")

VOCAB_SIZE = len(wordtoix) + 1
print(f"\n\nSet the   VOCAB_SIZE = len(wordtoix) + 1 = {VOCAB_SIZE}")

len(wordtoix) = 6757

CHECK : reloaded value = 6757


Set the   VOCAB_SIZE = len(wordtoix) + 1 = 6758


In [286]:
## see the index output by wordtoix for the start and end sequence tokens as well as some random one word
print( wordtoix.get('startseq') , wordtoix.get('endseq') , wordtoix.get('cat') )

1 9 526


In [287]:
print(f"Shape of embedding_matrix = {embedding_matrix.shape}")
print(f"\nCHECK : reloaded shape = 6758,200")

Shape of embedding_matrix = (6758, 200)

CHECK : reloaded shape = 6758,200


In [288]:
## Recheck these parameters that will define the Decoder architecture ::: EMBEDDING_DIMS , VOCAB_SIZE , MAX_LENGTH_CAPTION
print(f"SHOULD be:\nEMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49")
print(f"\nThe values to be used in Decoder setup:\nEMBEDDING_DIMS = {EMBEDDING_DIMS} , VOCAB_SIZE = {VOCAB_SIZE} , MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")

SHOULD be:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49

The values to be used in Decoder setup:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49


## ALWAYS - Define the RNN Decoder model
## Use the GloVe embeddings matrix values to update layer weights and freeze ONLY those weights


### About Keras Emebedding layer
### 
#### Default initializing of this layer: https://keras.io/api/layers/core_layers/embedding/
######tf.keras.layers.Embedding(
######    input_dim, output_dim,
######    embeddings_initializer="uniform",
######    embeddings_regularizer=None,
######    activity_regularizer=None,
######    embeddings_constraint=None,
######    mask_zero=False,
######    input_length=None,
######    **kwargs
######)

In [291]:
print(f"\nThe values to be used in Decoder setup:\nEMBEDDING_DIMS = {EMBEDDING_DIMS} , VOCAB_SIZE = {VOCAB_SIZE} , MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")


The values to be used in Decoder setup:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49


In [292]:
## Decoder Model defining

## parameters to define model - all these three populated earlier
## EMBEDDING_DIMS , VOCAB_SIZE , MAX_LENGTH_CAPTION

inputs1 = keras.Input(shape=(2048,))
fe1 = keras.layers.Dropout(0.5)(inputs1)
fe2 = keras.layers.Dense(256, activation='relu')(fe1)

# partial caption sequence model
inputs2 = keras.Input(shape=(MAX_LENGTH_CAPTION,))
se1 = keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIMS, mask_zero=True)(inputs2)
se2 = keras.layers.Dropout(0.5)(se1)
se3 = keras.layers.LSTM(256)(se2)

# decoder (feed forward) model
decoder1 = keras.layers.add([fe2, se3])
decoder2 = keras.layers.Dense(256, activation='relu')(decoder1)
outputs = keras.layers.Dense(VOCAB_SIZE, activation='softmax')(decoder2)

# merge the two input models
model_RNN_decoder = keras.models.Model(inputs=[inputs1, inputs2], outputs=outputs)

In [293]:
## Output of command:  model_RNN_decoder.summary() BEFORE FREEZING THE EMBEDDINGS WEIGHTS
## NOTE the count of Trainable params
model_RNN_decoder.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 49)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 49, 200)      1351600     input_4[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 2048)         0           input_3[0][0]                    
_______________________________________________________________________________________

In [ ]:
## Output of command:  model_RNN_decoder.summary() BEFORE FREEZING THE EMBEDDINGS WEIGHTS
## NOTE the count of Trainable params

#Model: "functional_3"
#__________________________________________________________________________________________________
#Layer (type)                    Output Shape         Param #     Connected to                     
#==================================================================================================
#input_4 (InputLayer)            [(None, 49)]         0                                            
#__________________________________________________________________________________________________
#input_3 (InputLayer)            [(None, 2048)]       0                                            
#__________________________________________________________________________________________________
#embedding_1 (Embedding)         (None, 49, 200)      1351600     input_4[0][0]                    
#__________________________________________________________________________________________________
#dropout_2 (Dropout)             (None, 2048)         0           input_3[0][0]                    
#__________________________________________________________________________________________________
#dropout_3 (Dropout)             (None, 49, 200)      0           embedding_1[0][0]                
#__________________________________________________________________________________________________
#dense_3 (Dense)                 (None, 256)          524544      dropout_2[0][0]                  
#__________________________________________________________________________________________________
#lstm_1 (LSTM)                   (None, 256)          467968      dropout_3[0][0]                  
#__________________________________________________________________________________________________
#add_1 (Add)                     (None, 256)          0           dense_3[0][0]                    
#                                                                 lstm_1[0][0]                     
#__________________________________________________________________________________________________
#dense_4 (Dense)                 (None, 256)          65792       add_1[0][0]                      
#__________________________________________________________________________________________________
#dense_5 (Dense)                 (None, 6758)         1736806     dense_4[0][0]                    
#==================================================================================================
#Total params: 4,146,710
#Trainable params: 4,146,710
#Non-trainable params: 0
#__________________________________________________________________________________________________

In [294]:
OPDIR

'../working/'

In [ ]:
#RNN_decoder_model_plot_filename = r'RNN_decoder_model_plot_97kTrain_1.jpg'
#tf.keras.utils.plot_model(model_RNN_decoder, to_file= OPDIR + RNN_decoder_model_plot_filename, show_shapes=True, show_layer_names=True)

### ALWAYS - set weights using the GloVe embedding matrix and freeze it

In [295]:
## contain the 200 dim representation of our vocab_threshold based output words and the 0-pad value which was left as 0's
embedding_matrix.shape

(6758, 200)

In [296]:
model_RNN_decoder.layers[2].set_weights([embedding_matrix])
model_RNN_decoder.layers[2].trainable = False

### DO ONCE - RNN Decoder summary and plot
#### AFTER loading GloVe embeddings and Freezing

In [297]:
## Output of command:  model_RNN_decoder.summary() AFTER FREEZING THE EMBEDDINGS WEIGHTS
## NOTE the count of Trainable params has REDUCED
model_RNN_decoder.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 49)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 49, 200)      1351600     input_4[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 2048)         0           input_3[0][0]                    
_______________________________________________________________________________________

In [ ]:
## Output of command:  model_RNN_decoder.summary() AFTER FREEZING THE EMBEDDINGS WEIGHTS
## NOTE the count of Trainable params has REDUCED

#Model: "functional_3"
#__________________________________________________________________________________________________
#Layer (type)                    Output Shape         Param #     Connected to                     
#==================================================================================================
#input_4 (InputLayer)            [(None, 49)]         0                                            
#__________________________________________________________________________________________________
#input_3 (InputLayer)            [(None, 2048)]       0                                            
#__________________________________________________________________________________________________
#embedding_1 (Embedding)         (None, 49, 200)      1351600     input_4[0][0]                    
#__________________________________________________________________________________________________
#dropout_2 (Dropout)             (None, 2048)         0           input_3[0][0]                    
#__________________________________________________________________________________________________
#dropout_3 (Dropout)             (None, 49, 200)      0           embedding_1[0][0]                
#__________________________________________________________________________________________________
#dense_3 (Dense)                 (None, 256)          524544      dropout_2[0][0]                  
#__________________________________________________________________________________________________
#lstm_1 (LSTM)                   (None, 256)          467968      dropout_3[0][0]                  
#__________________________________________________________________________________________________
#add_1 (Add)                     (None, 256)          0           dense_3[0][0]                    
#                                                                 lstm_1[0][0]                     
#__________________________________________________________________________________________________
#dense_4 (Dense)                 (None, 256)          65792       add_1[0][0]                      
#__________________________________________________________________________________________________
#dense_5 (Dense)                 (None, 6758)         1736806     dense_4[0][0]                    
#==================================================================================================
#Total params: 4,146,710
#Trainable params: 2,795,110
#Non-trainable params: 1,351,600
#__________________________________________________________________________________________________

In [ ]:
#tf.keras.utils.plot_model(model_RNN_decoder, to_file=OPDIR+'RNN_decoder_model_plot_AFTER_GLOVE_1.jpg', show_shapes=True, show_layer_names=True)

### ALWAYS - Compile RNN model

In [298]:
## setup the optimizer and compile

## see the default LR used, set some dummy to check it works
optimizer_adam = tf.keras.optimizers.Adam()  ## if nothing specified the default LR = 0.001
model_RNN_decoder.compile(loss='categorical_crossentropy', optimizer=optimizer_adam)
print(f"Default LR without explicity setting so far = {model_RNN_decoder.optimizer.learning_rate.numpy()}")

optimizer_adam.learning_rate.assign(0.5)
print(f"After setting LR as 0.5 = {model_RNN_decoder.optimizer.learning_rate.numpy()}")

Default LR without explicity setting so far = 0.0010000000474974513
After setting LR as 0.5 = 0.5


## ALWAYS - Train the RNN Decoder

In [299]:
OPDIR_WEIGHTS

'../working/weights_out/'

In [300]:
os.listdir( OPDIR_WEIGHTS )

[]

In [302]:
## values setup earlier - for reference

print(f"\tEncodings data:\nlen(img_encodings_train) = {len(img_encodings_train)}\nlen(img_encodings_val) = {len(img_encodings_val)}")
print(f"\tDescriptions data:\nlen(descriptions_train) = {len(descriptions_train)}\nlen(descriptions_val) = {len(descriptions_val)}")

print(f"\nNo Validation Set being used for now.\n")

print(f"len(wordtoix) = {len(wordtoix)}")
print(f"VOCAB_SIZE = {VOCAB_SIZE}")
print(f"EMBEDDING_DIMS = {EMBEDDING_DIMS}")
print(f"embedding_matrix Shape = {embedding_matrix.shape}")
print(f"MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")

	Encodings data:
len(img_encodings_train) = 97000	len(img_encodings_val) = 3000
	Descriptions data:
len(descriptions_train) = 97000	len(descriptions_val) = 3000

No Validation Set being used for now.

len(wordtoix) = 6757
VOCAB_SIZE = 6758
EMBEDDING_DIMS = 200
embedding_matrix Shape = (6758, 200)
MAX_LENGTH_CAPTION = 49


In [ ]:
MODEL_RUN_NUMBER = 2 ## large training data - 97k train
MODEL_RUN_NUMBER

In [ ]:
## For reference: def data_generator(_descriptions, _imgs_features_arr, _wordtoix, _max_length, _images_per_batch):
## Notes:
##      1) The data generator function needs exactly the CORRECT descriptions for the training (or validation data). It takes the keys from the descriptions and then
##         searches the image encodings array. So it is fine even if we use an image encodings with more images.
##         But we already made the dedicated arrays for Train and Validation datasets, so use them explicity!!!

########### Phase 1 #########

print(f"\n\nMODEL {MODEL_RUN_NUMBER} :: Training Phase 1 started at :: {datetime.datetime.now().strftime('%H:%M:%S')}\n")

LR_1 = 0.0005
BATCH_SIZE_1 = 128   ## how many images per batch
N_EPOCHS_1 = 2
STEPS_PER_EPOCH_1 = len(descriptions_train) // BATCH_SIZE_1
print(f"\nPhase 1 parameters:")
print(f"STEPS_PER_EPOCH_1 = {STEPS_PER_EPOCH_1}")
print(f"BATCH_SIZE_1 = {BATCH_SIZE_1}")
print(f"N_EPOCHS_1 = {N_EPOCHS_1}")

optimizer_adam.learning_rate.assign(LR_1)

for i in range(N_EPOCHS_1):
    print(f"\nEpoch {i+1} started at {datetime.datetime.now().strftime('%H:%M:%S')}\nLR used = {model_RNN_decoder.optimizer.learning_rate.numpy()} \n")
    generator_1 = data_generator_1(descriptions_train, img_encodings_train, wordtoix, MAX_LENGTH_CAPTION, BATCH_SIZE_1, VOCAB_SIZE)
    model_RNN_decoder.fit_generator(generator_1, epochs=1, steps_per_epoch=STEPS_PER_EPOCH_1, verbose=1)
    
    model_RNN_decoder.save_weights( OPDIR_WEIGHTS + 'Decoder_Run_' + str(MODEL_RUN_NUMBER) + '_Wt_ep_' + str(i+1) + '.h5' )

In [ ]:
## Console output after training:

MODEL 2 :: Training Phase 1 started at :: 17:15:11


Phase 1 parameters:
STEPS_PER_EPOCH_1 = 757
BATCH_SIZE_1 = 128
N_EPOCHS_1 = 2

Epoch 1 started at 17:15:11
LR used = 0.0005000000237487257 

757/757 [==============================] - 7705s 10s/step - loss: 4.3657

Epoch 2 started at 19:23:55
LR used = 0.0005000000237487257 

757/757 [==============================] - 7793s 10s/step - loss: 3.4819


In [ ]:
#model_reset_states
#lr optimizer
#tuner